## Importing Libraries

In [1]:
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import pickle
from scipy.sparse import hstack

Using TensorFlow backend.


## Creating Model Class

In [2]:
class agegroup():
  def __init__(self):
        self.model_events = pickle.load(open("xgb_model_events.pkl", "rb")) 
        self.model_noevents = pickle.load(open("xgb_model_noevents.pkl", "rb")) 
        
        self.noevents_brand = pickle.load(open("noevents_vectorizers/brand_vec.pkl", "rb")) 
        self.noevents_model = pickle.load(open("noevents_vectorizers/model_vec.pkl", "rb")) 
        self.noevents_y = pickle.load(open("noevents_vectorizers/y_encoder.pkl", "rb")) 
        
        self.events_brand = pickle.load(open("events_vectorizers/brand_vec.pkl", "rb")) 
        self.events_model = pickle.load(open("events_vectorizers/model_vec.pkl", "rb")) 
        self.events_installed = pickle.load(open("events_vectorizers/installed_vec.pkl", "rb")) 
        self.events_active = pickle.load(open("events_vectorizers/active_vec.pkl", "rb")) 
        self.events_y = pickle.load(open("events_vectorizers/y_encoder.pkl", "rb")) 
        
            
  def __determine_model(self):
        if self.inputs.shape[0]==14:
            self.model = self.model_noevents
        else:
            self.model = self.model_events
                
  def __create_feature_vector(self):
        if self.inputs.shape[0]==14:
            self.x = hstack((self.noevents_brand.transform([self.inputs['brand']]), self.noevents_model.transform([self.inputs['model']]), 
                             self.inputs['screen_size'], self.inputs['ram_gb'], self.inputs['camera'], self.inputs['specs_available'], 
                             self.inputs['release_bin'], self.inputs['female_pred'], self.inputs['male_pred'],
                             self.inputs['pred_age']))
        else:
            self.x = hstack((self.events_brand.transform([self.inputs['brand']]), self.events_model.transform([self.inputs['model']]), 
                             self.events_installed.transform([self.inputs['installed_apps_string']]), self.events_active.transform([self.inputs['active_apps_string']]),  
                             self.inputs['screen_size'], self.inputs['ram_gb'], self.inputs['camera'], self.inputs['specs_available'], 
                             self.inputs['release_bin'], self.inputs['female_pred'], self.inputs['male_pred'], self.inputs['pred_age'], 
                             self.inputs['num_travels'], self.inputs['mean_latitude'], self.inputs['mean_longitude'], self.inputs['app_usage'], 
                             self.inputs['app_usage_session'], self.inputs['activity_hour'], self.inputs['activity_day'], self.inputs['installed_app_onehot'], 
                             self.inputs['active_app_onehot'], self.inputs['active_app_usage'], self.inputs['location_available'])) 
        
                        
  def predict(self, inputs):
        self.inputs = inputs
        self.__determine_model()
        self.__create_feature_vector()
        
        self.y = self.model.predict(self.x)
        
        if self.inputs.shape[0]==14:
            print(self.noevents_y.inverse_transform(self.y)[0])
        else:
            print(self.events_y.inverse_transform(self.y)[0])

## Loading the Dataset to obtain test data

In [5]:
df_noevents = pd.read_pickle("eval_data/noevents_test.pkl")
df_events = pd.read_pickle("eval_data/events_test.pkl")

In [6]:
test_event = df_events.loc[40174]
test_noevent = df_noevents.loc[29491]

In [7]:
model = agegroup()

## Prediction

In [8]:
model.predict(test_noevent)

M23-26


In [9]:
model.predict(test_event)

M22-
